# Inicializar ODM

In [1]:
import ODM as ODM

ODM.initApp(db_name="Practica2")

# Ejercicio 1

Listado de todas personas que han estudiado en la UPM o UAM.

`$unwind` → Separar múltiples estudios

`$match` → Filtrar por condición

In [25]:
pipeline_1 = [
    {"$unwind": "$estudios"},
    {"$match": {"estudios.universidad": {"$in": ["UPM", "UAM"]}}},
    {"$project": {"_id": 0, "nombre": 1, "estudios": 1}}
]

for persona in ODM.Persona.aggregate(pipeline_1):
    print(persona)

{'nombre': 'Javier Moreno', 'estudios': {'universidad': 'UPM', 'titulo': 'Grado en Informática', 'anio_fin': 2013}}
{'nombre': 'Carlos García', 'estudios': {'universidad': 'UPM', 'titulo': 'Grado en Informática', 'anio_fin': 2018}}
{'nombre': 'David Ruiz', 'estudios': {'universidad': 'UAM', 'titulo': 'Máster en IA', 'anio_fin': 2021}}
{'nombre': 'Lucía Martínez', 'estudios': {'universidad': 'UAM', 'titulo': 'Grado en Matemáticas', 'anio_fin': 2017}}
{'nombre': 'Laura Sánchez', 'estudios': {'universidad': 'UAM', 'titulo': 'Grado en Telecomunicaciones', 'anio_fin': 2015}}


# Ejercicio 2

Listado de las diferentes universidades en las que han estudiado las personas
residentes en Madrid.

`$unwind` → Separamos múltiples estudios

`$match` → Filtrar por Madrid

`$group` → Agrupar por Universidad

In [26]:
pipeline_2 = [
    {"$unwind": "$estudios"},
    {"$match": {"ciudad_residencia": "Madrid"}},
    {"$group": {"_id": "$estudios.universidad"}},
    {"$project": {"_id": 0, "universidad": "$_id"}}
]

for universidad in ODM.Persona.aggregate(pipeline_2):
    print(universidad)

{'universidad': 'UCM'}
{'universidad': 'Complutense'}
{'universidad': 'UPM'}
{'universidad': 'UOC'}
{'universidad': 'UAM'}
{'universidad': 'UC3M'}


# Ejercicio 3

Personas que, en la descripción de su perfil, incluye los términos "Big Data" o
"Inteligencia Artificial"

`$match` + `$regex` → Expresiones regulares para filtrar la condición

In [27]:
pipeline_3 = [
    {"$match": {"descripcion": {"$regex": "Big Data|Inteligencia Artificial", "$options": "i"}}},
    {"$project": {"_id": 0, "nombre": 1, "descripcion": 1}}
]

for persona in ODM.Persona.aggregate(pipeline_3):
    print(persona)

{'nombre': 'Sara Díaz', 'descripcion': 'Experiencia en sistemas distribuidos y microservicios. Trabajo con Big Data a gran escala.'}
{'nombre': 'Elena Álvarez', 'descripcion': 'Experiencia en sistemas distribuidos y microservicios. Investigación en Inteligencia Artificial aplicada.'}
{'nombre': 'Carlos García', 'descripcion': 'Apasionado por el desarrollo backend en Python. Trabajo con Big Data a gran escala.'}
{'nombre': 'Pablo López', 'descripcion': 'Interés en analítica de datos y visualización. Trabajo con Big Data a gran escala.'}
{'nombre': 'David Ruiz', 'descripcion': 'Experto en bases de datos NoSQL y mensajería. Investigación en Inteligencia Artificial aplicada.'}
{'nombre': 'Lucía Martínez', 'descripcion': 'Experiencia en sistemas distribuidos y microservicios. Investigación en Inteligencia Artificial aplicada.'}
{'nombre': 'Laura Sánchez', 'descripcion': 'Experto en bases de datos NoSQL y mensajería. Investigación en Inteligencia Artificial aplicada.'}


# Ejercicio 4

Guarda en una tabla nueva el listado de las personas que ha terminado alguno
de sus estudios en el 2017 o después.

No usamos `$unwind` porque este operador **separa el array `estudios` en varios documentos (uno por cada estudio)**.  
Eso provocaría que una misma persona apareciese varias veces y que el operador `$out` diera error al intentar insertar documentos con el mismo `_id`.

Por tanto, al **no usar `$unwind`**, cada persona se mantiene como un único documento, y el filtro:
```json
{"estudios.anio_fin": {"$gte": 2017}}
```

selecciona correctamente a quienes tienen al menos un estudio terminado a partir de 2017.

El segundo pipeline (`pipeline_4_1`) solo se utiliza para mostrar los años de finalización de los estudios de cada persona y confirmar visualmente que los resultados son correctos, aunque el enunciado no lo pida.

In [28]:
pipeline_4 = [
    {"$match": {"estudios.anio_fin": {"$gte": 2017}}},
    {"$out": "Graduados_2017"}
]

pipeline_4_1 = [
    {"$match": {"estudios.anio_fin": {"$gte": 2017}}},
    {"$group": {"_id": "$nombre", "anios_estudios": {"$push": "$estudios.anio_fin"}}},
    {"$project": {"_id": 1, "anios_estudios": 1}}
]

ODM.Persona.aggregate(pipeline_4)

for persona in ODM.Persona.aggregate(pipeline_4_1):
    print(persona)

{'_id': 'Laura Sánchez', 'anios_estudios': [[2015, 2017]]}
{'_id': 'Carlos García', 'anios_estudios': [[2018, 2020]]}
{'_id': 'Miguel Gómez', 'anios_estudios': [[2017, 2019]]}
{'_id': 'Sara Díaz', 'anios_estudios': [[2019, 2021]]}
{'_id': 'Elena Álvarez', 'anios_estudios': [[2018, 2020]]}
{'_id': 'David Ruiz', 'anios_estudios': [[2021]]}
{'_id': 'Pablo López', 'anios_estudios': [[2016, 2019]]}
{'_id': 'Lucía Martínez', 'anios_estudios': [[2017]]}


# Ejercicio 5

Calcular el número medio de estudios realizados por las personas que han
trabajado o trabajan en la Microsoft.

En resumen:
- `$match` → selecciona empleados de Microsoft.  
- `$project` → cuenta los estudios de cada persona.  
- `$group` → calcula la media total de estudios.

In [ ]:
pipeline_5 = [
    {"$match": {"trabajos.empresa": "Microsoft"}},
    {"$project": {"num_estudios": {"$size": "$estudios"}}},
    {"$group": {"_id": "Media_Estudios" , "media_estudios": {"$avg": "$num_estudios"}}}
]

print(list(ODM.Persona.aggregate(pipeline_5)))

[{'_id': 'Media_Estudios', 'media_estudios': 1.6666666666666667}]


# Ejercicio 6

Distancia media al trabajo (distancia geodésica) de los actuales trabajadores
de Google. Se pueden indicar las coordenadas de la oficina de Google
manualmente.

`$geoNear` → Sacar la distancia de cada registro con Google

`$match` → Filtramos por la condición

`$group` → Calculamos la media y agrupamos

In [3]:
google_coordinates = {"type":"Point","coordinates":[-3.6923, 40.4460]}

pipeline_6 = [{"$geoNear": {"near": google_coordinates, "key": "domicilio_loc", "distanceField": "dist_m"}},
            {"$match": {"trabajos": {"$elemMatch": {"empresa": "Google", "fin": None}}}},
            {"$group": {"_id": None, "distancia_media_m": {"$avg": "$dist_m"}, "empleados": {"$sum": 1}}},
            {"$project": {"_id":0, "distancia_media_km": {"$divide": ["$distancia_media_m", 1000]}, "empleados":1}}
]

print(list(ODM.Persona.aggregate(pipeline_6)))

[{'empleados': 3, 'distancia_media_km': 199.73270842422883}]


# Ejercicio 7

Listado de las tres universidades que más veces aparece como centro de
estudios de las personas registradas. Mostrar universidad y el número de
veces que aparece.

`$unwind` → Separamos múltiples estudios

`$group` → Agrupamos por universidad y contamos el número de veces que aparece

`$sort` → Orden de mayor a menor

`$limit` → Limitar a 3

In [31]:
pipeline_7 = [
    {"$unwind": "$estudios"},
    {"$group": {"_id": "$estudios.universidad", "veces": {"$sum": 1}}},
    {"$sort": {"veces": -1}},
    {"$limit": 3},
    {"$project": {"_id": 0, "universidad": "$_id", "veces": 1}}
]

for universidad in ODM.Persona.aggregate(pipeline_7):
    print(universidad)

{'veces': 3, 'universidad': 'UAM'}
{'veces': 3, 'universidad': 'UPV'}
{'veces': 2, 'universidad': 'UOC'}
